In [6]:
import os
import sys
import pandas as pd

In [7]:
sys.path.append("/home/rocassius/w266_final/scripts/assembly")
sys.path.append("/home/rocassius/w266_final/scripts/modeling")

In [11]:
from document import load_documents
from constant import DOC_PRAYER_PATH, DOCUMENT, MIN_SESSION, MAX_SESSION
sessions = list(range(MIN_SESSION, MAX_SESSION+1))

In [26]:
from token_mapping import *
from helper import *
#from embeddings import *

In [27]:
docs_df = load_documents([111], DOC_PRAYER_PATH)

In [31]:
docs_df['subject'].unique()

array(['abortion', 'drug_alcohol', 'budget', 'guns', 'defense', 'economy',
       'education', 'foreign', 'health', 'immigration', 'labor',
       'minorities', 'women', 'tax', 'trade'], dtype=object)

In [ ]:
docs_df.shape

In [ ]:
docs_df

In [8]:
docs_df.dtypes

speakerid    object
lastname     object
firstname    object
chamber      object
state        object
gender       object
party        object
document     object
subject      object
session      object
dtype: object

In [218]:
feature_columns = [
 'speakerid',
 'chamber',
 'state',
 'gender',
 'party',
 'session',
 'subject']


In [10]:
import os
import pickle

import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences


#os.chdir("../assembly/")
from constant import EMBEDDINGS


# constants
GLOVE_DIMS = [50, 100, 200, 300]
EMBEDDING_DIM = GLOVE_DIMS[0]
GLOVE_PATH = os.path.join(EMBEDDINGS, 'glove6B/glove.6B.%dd.csv' % EMBEDDING_DIM)


def fetch_embeddings(embeddings_dim = EMBEDDING_DIM):
    
    path = os.path.join(EMBEDDINGS, 'glove6B/glove.6B.%dd.csv' % embeddings_dim)
    embeddings_index = pd.read_csv(path).to_dict(orient = 'list')
    
    return embeddings_index



def build_embedding_matrix(word_index, embeddings_index, stopwords=[]):
    
    # get the embedding dimension
    embedding_dim = len(embeddings_index['the'])
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None and word not in stopwords:
            # words not found in embedding index and stopwords will be all-zeros.
            embedding_matrix[i] = embedding_vector 
            
    embedding_matrix = embedding_matrix.astype('float16')
    
    return embedding_matrix

In [43]:
def build_tokenizer_dict(document_df, num_words=None, max_span_len = MAX_TOKENS):
    """Returns a dictionary with useful properties of a tokenizer fit on document_df
    """
    tokenizer = Tokenizer(num_words)
    tokenizer.fit_on_texts(document_df['document'])
    tokenizer_pad = partial(tokenize_pad, tokenizer=tokenizer, max_span_len=max_span_len)
    
    tokenizer_dict = {
        'tokenizer': tokenizer, 
        'tokenize_pad': tokenizer_pad, 
        'word_index': tokenizer.word_index,
        'max_span_length': max_span_len}
    
    return tokenizer_dict

In [22]:
os.path.join(EMBEDDINGS, 'glove6B/glove.6B.%dd.csv' % 300)

'gs://rwc1/data/embeddings/glove6B/glove.6B.300d.csv'

In [28]:
import time 
start = time.time()

embedding_index = {}
with open("/home/rocassius/gen-data/tools/glove.6B.300d.csv", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embedding_index[word] = vector
    
end = time.time()
elapsed = end - start 
print(round(elapsed, 3))

2.469


In [29]:
len(embedding_index)

301

In [31]:
#df = pd.read_csv("/home/rocassius/gen-data/tools/glove.6B.300d.csv")

In [32]:
embedding_index = fetch_embeddings(embeddings_dim=100)

In [33]:
len(embedding_index)

400000

In [37]:
embedding_keys = list(embedding_index.keys())

In [23]:
from gensim import corpora, models

In [74]:
tokenizer_dict = build_tokenizer_dict(docs_df)

In [90]:
#tokenizer_dict1 = build_tokenizer_dict(docs_df.sample(100))

In [75]:
tokenizer = tokenizer_dict['tokenizer']

In [78]:
tokenizer.num_words = None

In [80]:
len(tokenizer.word_index)

904056

In [82]:
word_counts = pd.Series(tokenizer.word_counts).sort_values(ascending=False)

In [96]:
len(word_counts)

904056

In [100]:
word_counts[word_counts > 20]

the           14889254
of             8212410
to             6348117
and            5725461
in             4573531
                ...   
maga                21
cloverdale          21
garrigue            21
scotsman            21
dollarsfor          21
Length: 58088, dtype: int64

In [104]:
word_counts[word_counts <= 20]

kamehamehas      20
toyou            20
warwar           20
renourishment    20
citizenand       20
                 ..
ringle            1
whatevery         1
inedicare         1
selfhelpthat      1
straussdraper     1
Length: 845968, dtype: int64

In [101]:
np.sum(word_counts[word_counts < 20])

1827433

In [102]:
np.sum(word_counts[word_counts >= 20])

194252154

In [110]:
# for i in pd.DataFrame(word_counts[word_counts >= 20][500:]).sample(100).itertuples(): print(i)

In [105]:
np.sum(word_counts[word_counts >= 20][500:])

58144436

In [111]:
from preprocess import *
stopwords = english_stopwords + name_stopwords

In [112]:
len(english_stopwords)

179

Index(['the', 'of', 'to', 'and', 'in', 'a', 'that', 'is', 'i', 'for',
       ...
       'criminalsapparently', 'myrannual', 'opzens', 'unanimitythat',
       'sentimentalmost', 'ringle', 'whatevery', 'inedicare', 'selfhelpthat',
       'straussdraper'],
      dtype='object', length=904056)

In [118]:
intersect = np.intersect1d(english_stopwords, word_counts.index[:500])
intersect

array(['a', 'about', 'after', 'again', 'against', 'all', 'am', 'an',
       'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before',
       'being', 'between', 'both', 'but', 'by', 'did', 'do', 'does',
       'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further',
       'had', 'has', 'have', 'having', 'he', 'her', 'here', 'him', 'his',
       'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me',
       'more', 'most', 'my', 'no', 'not', 'of', 'off', 'on', 'only', 'or',
       'other', 'our', 'out', 'over', 'own', 's', 'same', 'she', 'should',
       'so', 'some', 'such', 'than', 'that', 'the', 'their', 'them',
       'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to',
       'too', 'under', 'until', 'up', 'very', 'was', 'we', 'were', 'what',
       'when', 'where', 'which', 'while', 'who', 'why', 'with', 'you',
       'your'], dtype=object)

In [119]:
len(intersect)

105

In [122]:
name_int = np.intersect1d(name_stopwords, word_counts.index)

In [123]:
len(name_int)

6178

In [126]:
len(np.intersect1d(name_stopwords, word_counts[word_counts >= 20].index))

4324

In [89]:
tokenizer_dict['tokenize_pad'](["hello my name is a rowan and I work in industry in health considered report river whose business"])

array([[22708,    42,   922,     8,     6, 20235,     4,     9,   122,
            5,   203,     5,   107,   658,   189,   864,   602,   165,
            0,     0]], dtype=int32)

In [64]:
# make tokenzer dict / numwords = 200000 (glove size)
# examine low frequency words
# remove low frequency words at certain threshold

# 

In [134]:
#tokenizer_dict
len(word_counts)

904056

In [142]:
word_counts.shape

(904056,)

In [144]:
word_counts[word_counts < 20].shape

(844516,)

In [148]:
tokenizer.word_index

{'the': 1,
 'of': 2,
 'to': 3,
 'and': 4,
 'in': 5,
 'a': 6,
 'that': 7,
 'is': 8,
 'i': 9,
 'for': 10,
 'this': 11,
 'it': 12,
 'we': 13,
 'on': 14,
 'have': 15,
 'as': 16,
 'be': 17,
 'are': 18,
 'not': 19,
 'by': 20,
 'with': 21,
 'from': 22,
 'was': 23,
 'our': 24,
 'has': 25,
 'which': 26,
 'or': 27,
 'an': 28,
 'they': 29,
 'at': 30,
 'would': 31,
 'he': 32,
 'bill': 33,
 'been': 34,
 'their': 35,
 'but': 36,
 'who': 37,
 'all': 38,
 'his': 39,
 'there': 40,
 'president': 41,
 'my': 42,
 'states': 43,
 'these': 44,
 'do': 45,
 'one': 46,
 'if': 47,
 'people': 48,
 'other': 49,
 'us': 50,
 'more': 51,
 'said': 52,
 'what': 53,
 'were': 54,
 'when': 55,
 'no': 56,
 'united': 57,
 'about': 58,
 'time': 59,
 'years': 60,
 'so': 61,
 'its': 62,
 'had': 63,
 'government': 64,
 'any': 65,
 'year': 66,
 'out': 67,
 'you': 68,
 'than': 69,
 'american': 70,
 'war': 71,
 'congress': 72,
 'country': 73,
 'some': 74,
 'state': 75,
 'those': 76,
 'budget': 77,
 'many': 78,
 'federal': 79,
 'am

In [145]:
len(embedding_index)

400000

In [139]:
np.sum(word_counts[word_counts <= 30])

2110957

In [ ]:
tokenizer

In [154]:
low_count_words = list(word_counts[word_counts < 20].index)

In [ ]:
high_countwords

In [159]:
"boring" in word_counts.index

True

In [179]:
stopwords = english_stopwords + name_stopwords

In [171]:
#stopwords

In [177]:
1 not in [12,1]

False

In [181]:
good_count_words = word_counts[100:][word_counts[100:] >= 20].index

In [182]:
len(good_count_words)

59440

In [192]:
good_count_words

Index(['great', 'legislation', 'education', 'first', 'foreign', 'up', 'health',
       'world', 'public', 'most',
       ...
       'yoakum', 'crosscurrents', 'titie', 'sherer', 'griefstricken',
       'misspending', 'reiche', 'israeliarab', 'noone', 'riddell'],
      dtype='object', length=59440)

In [183]:
good_words = [w for w in good_count_words if w not in stopwords]

In [188]:
good_count_glove_words = np.union1d(good_words, list(embedding_index.keys()))

In [189]:
good_glove_words = [w for w in good_count_glove_words if w not in stopwords]

In [204]:
"the" in good_words

False

In [205]:
"mine" in good_words

True

In [206]:
"you" in good_words

False

In [215]:
len(good_words)

55038

In [202]:
len(good_glove_words)

404053

In [195]:
# " ".join(good_words)

In [198]:
def build_tokenizer_dict(docs, num_words=None, max_span_len = MAX_TOKENS):
    """Returns a dictionary with useful properties of a tokenizer fit on document_df
    """
    tokenizer = Tokenizer(num_words)
    tokenizer.fit_on_texts(docs)
    tokenizer_pad = partial(tokenize_pad, tokenizer=tokenizer, max_span_len=max_span_len)
    
    tokenizer_dict = {
        'tokenizer': tokenizer, 
        'tokenize_pad': tokenizer_pad, 
        'word_index': tokenizer.word_index,
        'max_span_length': max_span_len}
    
    return tokenizer_dict

In [201]:
tokenizer_dict_good = build_tokenizer_dict([" ".join(good_words)])
tokenizer_dict_good

{'tokenizer': <keras_preprocessing.text.Tokenizer at 0x7fd026fa6110>,
 'tokenize_pad': functools.partial(<function tokenize_pad at 0x7fd12af59950>, tokenizer=<keras_preprocessing.text.Tokenizer object at 0x7fd026fa6110>, max_span_len=20),
 'word_index': {'great': 1,
  'legislation': 2,
  'education': 3,
  'first': 4,
  'foreign': 5,
  'health': 6,
  'world': 7,
  'public': 8,
  'percent': 9,
  'upon': 10,
  'think': 11,
  'last': 12,
  'make': 13,
  'support': 14,
  'million': 15,
  'work': 16,
  'must': 17,
  'trade': 18,
  'could': 19,
  'every': 20,
  'service': 21,
  'labor': 22,
  'well': 23,
  'know': 24,
  'department': 25,
  'military': 26,
  'union': 27,
  'defense': 28,
  'economy': 29,
  'school': 30,
  'want': 31,
  'care': 32,
  'programs': 33,
  'like': 34,
  'important': 35,
  'believe': 36,
  'much': 37,
  'take': 38,
  'fact': 39,
  'billion': 40,
  'members': 41,
  'pay': 42,
  'two': 43,
  'need': 44,
  'nation': 45,
  'administration': 46,
  'business': 47,
  'natio

In [207]:
tokenizer_dict_good_glove = build_tokenizer_dict([" ".join(good_glove_words)])
tokenizer_dict_good_glove

{'tokenizer': <keras_preprocessing.text.Tokenizer at 0x7fd0285c4490>,
 'tokenize_pad': functools.partial(<function tokenize_pad at 0x7fd12af59950>, tokenizer=<keras_preprocessing.text.Tokenizer object at 0x7fd0285c4490>, max_span_len=20),
 'word_index': {'1': 1,
  '2': 2,
  '3': 3,
  '4': 4,
  '000': 5,
  '5': 6,
  '6': 7,
  'com': 8,
  '7': 9,
  '8': 10,
  '9': 11,
  '0': 12,
  '10': 13,
  '12': 14,
  'non': 15,
  '15': 16,
  '11': 17,
  'a': 18,
  '20': 19,
  '16': 20,
  '18': 21,
  '50': 22,
  '13': 23,
  '30': 24,
  '14': 25,
  '17': 26,
  '25': 27,
  '19': 28,
  '40': 29,
  's': 30,
  'www': 31,
  '22': 32,
  '21': 33,
  '24': 34,
  'e': 35,
  '45': 36,
  '32': 37,
  '35': 38,
  'based': 39,
  'year': 40,
  '28': 41,
  '26': 42,
  '23': 43,
  '60': 44,
  '27': 45,
  '31': 46,
  '29': 47,
  '42': 48,
  '34': 49,
  '38': 50,
  '33': 51,
  '55': 52,
  '36': 53,
  '70': 54,
  '44': 55,
  're': 56,
  '37': 57,
  '39': 58,
  '75': 59,
  '43': 60,
  '52': 61,
  '48': 62,
  '41': 63,
  '4

In [128]:
def build_embedding_matrix(word_index, embeddings_index, stopwords=[]):
    
    # get the embedding dimension
    embedding_dim = len(embeddings_index['the'])
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None and word not in stopwords:
            # words not found in embedding index and stopwords will be all-zeros.
            embedding_matrix[i] = embedding_vector 
            
    embedding_matrix = embedding_matrix.astype('float16')
    
    return embedding_matrix

In [131]:
embedding_mat = build_embedding_matrix(tokenizer_dict['word_index'], embedding_index)

In [208]:
embedding_mat_good = build_embedding_matrix(tokenizer_dict_good['word_index'], embedding_index)

In [214]:
embedding_mat_good.shape

(55039, 100)

In [210]:
embedding_mat_good_glove = build_embedding_matrix(tokenizer_dict_good_glove['word_index'], embedding_index)

In [212]:
embedding_mat_good_glove.shape

(344310, 100)

In [216]:
docs_df

,speakerid,lastname,firstname,chamber,state,gender,party,document,subject,session
0,43045350,WHEELER,WILLIAM,H,NY,M,R,our federal government rusts we have no such t...,abortion,43
1,43043930,WOODFORD,STEWART,H,NY,M,R,restore confidence his grand scheme to return ...,abortion,43
2,43044540,HURLBUT,STEPHEN,H,IL,M,R,efforts made to secure power and plunder by th...,abortion,43
3,43045260,WALLACE,ALEXANDER,H,SC,M,R,should lhave a contraction of the currency is ...,abortion,43
4,43040351,SPRAGUE,WILLIAM,S,RI,M,L,making affidavit supported by his physician th...,abortion,43
...,...,...,...,...,...,...,...,...,...,...
92749,111120531,MIKULSKI,BARBARA,S,MD,F,D,of a more accurate food facility registry impr...,trade,111
92750,111120170,LATTA,ROBERT,H,OH,M,R,their lives to save others that day these meda...,trade,111
92751,111116441,HUTCHISON,KAY,S,TX,F,R,investment this year the development assistanc...,trade,111
92752,111120860,POLIS,JARED,H,CO,M,D,economytoo much in the last decade of folks pa...,trade,111


In [219]:
metadata_dict = build_metadata_dict(docs_df, feature_columns)

In [221]:
pickle_object(metadata_dict, os.path.join("/home/rocassius/gen-data/tools/prayer_tools/", "metadata_dict"))

In [25]:
pickle_object(tokenizer_dict_good, os.path.join("/home/rocassius/gen-data/tools/prayer_tools/", "tokenizer_dict"))


In [223]:
pickle_object(tokenizer_dict_good_glove, 
              os.path.join("/home/rocassius/gen-data/tools/prayer_tools/", "tokenizer_dict_wg"))

In [224]:
pickle_object(embedding_mat_good, 
              os.path.join("/home/rocassius/gen-data/tools/prayer_tools/", "embedding_matrix"))

In [225]:
pickle_object(embedding_mat_good_glove, 
              os.path.join("/home/rocassius/gen-data/tools/prayer_tools/", "embedding_matrix_wg"))

In [236]:
np.sum(tokenizer_dict_good['tokenizer'].texts_to_matrix(["this is the constantly run abortion and all "], mode="tfidf")[0])

1.2163953243244932

In [237]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_df=)

Fixing the tokenizer dict

In [15]:
prayer_tools_path = "/home/rocassius/gen-data/tools/prayer_tools"

tokenizer_dict = load_pickled_object(os.path.join(prayer_tools_path, "tokenizer_dict"))

In [18]:
tokenizer_dict['max_span_length'] = 40

In [22]:
tokenizer_dict['tokenize_pad'] = tokenizer_pad

In [24]:
tokenizer_dict['tokenize_pad']("Hello my name is rowan")[0].shape

(40,)

In [21]:
tokenizer_pad = partial(tokenize_pad, tokenizer=tokenizer_dict['tokenizer'], max_span_len=40)